In [37]:
import pandas as pd
import re
#f=open("./dialogue_key_value/kvret_train_public.json")
df=pd.read_json("kvret_train_public.json")
#df=pd.read_json("kvret_test_public.json")

In [30]:
#Knowledgebase cretaion
import pandas as pd
#f=open("./dialogue_key_value/kvret_train_public.json")
#df=pd.read_json("./dialogue_key_value/kvret_train_public.json")
colnames=[]
for i in range(len(df)):
    colnames.append(df["scenario"][i]['kb']['column_names'])
#tuples for drive
#tuples_d=[('poi','address','val'),('poi','poi_type','val'),('poi','traffic_info','val'),('poi','distance','val')]
tuples_d=[('poi','address','val'),('poi','traffic_info','val'),('poi','distance','val')]
#tuples for calendar
tuples_c=[('event','time','val'),('event','room','val'),('event','party','val'),('event','agenda','val'),('event','date','val')]
#tuples for weather
tuples_w=[('location','monday','val'),('location','tuesday','val'),('location','wednesday','val'),
          ('location','thursday','val'),('location','friday','val'),('location','saturday','val'),
          ('location','sunday','val')]
kb=[]
for i in range(len(df)):
    #print(i)
    kb_i=[]
    if df['scenario'][i]['kb']['items']:
        if(df['scenario'][i]['kb']['column_names'][0]=="poi"):
            for j in range(len(df['scenario'][i]['kb']['items'])):
                for k in range(len(tuples_d)):
                    tup=[]
                    tup.append(df['scenario'][i]['kb']['items'][j][tuples_d[k][0]])
                    tup.append(tuples_d[k][1])
                    tup.append(df['scenario'][i]['kb']['items'][j][tuples_d[k][1]])
                    kb_i.append(tup)
            pass
        elif(df['scenario'][i]['kb']['column_names'][0]=="event"):
            for j in range(len(df['scenario'][i]['kb']['items'])):
                #tup=[]
                for k in range(len(tuples_c)):
                    tup=[]
                    tup.append(df['scenario'][i]['kb']['items'][j][tuples_c[k][0]])
                    tup.append(tuples_c[k][1])
                    tup.append(df['scenario'][i]['kb']['items'][j][tuples_c[k][1]])
                    kb_i.append(tup)
            pass
        else:
            for j in range(len(df['scenario'][i]['kb']['items'])):
                tup=[]
                for k in range(len(tuples_w)):
                    tup=[]
                    tup.append(df['scenario'][i]['kb']['items'][j][tuples_w[k][0]] )
                    tup.append(tuples_w[k][1])
                    tup.append(df['scenario'][i]['kb']['items'][j][tuples_w[k][1]])
                    kb_i.append(tup)
            pass
    kb.append(kb_i)

In [31]:
x={'subject':[],'relation':[],'object':[]}
for i in kb:
    for j in i:
        x['subject'].append(j[0])
        x['object'].append(j[2])
        x['relation'].append(j[1])
x=pd.DataFrame(x)
x.drop_duplicates(inplace=True)
x.to_csv("kbtuples.csv")
nkb={}
nkb['relation']=x['relation']
nkb['subject']=x['subject']
nkb['object']=x['subject']+"_"+x['relation']
nkb=pd.DataFrame(nkb)
nkb.drop_duplicates(inplace=True)
nkb.to_csv('normalised_kbtuples.csv')
print(len(nkb))
#Canonical representations
objects=[]
for kb_i in kb:
    for ki in kb_i:
        objects.append('_'.join(ki[0].split(" "))+'_'+ki[1])

431


In [38]:
chats=[]
chats_complete=[]
for i in range(len(df)):
    chat=[]
    for j in range(len(df.iloc[i]["dialogue"])):
        chat.append(str(df.iloc[i]["dialogue"][j]["data"]["utterance"]).strip('"').lower())
        chats_complete.append(str(df.iloc[i]["dialogue"][j]["data"]["utterance"]).strip('"').lower())
    chats.append(chat)

In [33]:
#Preporcessing replacing values with their canonical representations
count=0
for i,(chat,kb_i) in enumerate(zip(chats,kb)):
    for j,ch in enumerate(chat):
        for ki in kb_i:
            if ki[0].lower() in ch:
                poi=ki[0].lower()
    for j,ch in enumerate(chat):
        #print(ch)
        for ki in kb_i:
            #print(ki)
            if ki[0].lower()==poi:
                #print(ki,ch)
                #print("hello")
                if 'day' in ki[1].lower():
                    for kki in ki[1].lower().split(","):
                        if kki in ch and ki[2].lower()!='home':
                            count=count+1
                            chats[i][j]=re.sub(kki,'_'.join(ki[0].split(" "))+'_'+ki[1],ch)
                if ki[2].lower() in ch and ki[2].lower()!='home':
                    count=count+1
                    chats[i][j]=re.sub(ki[2].lower(),'_'.join(ki[0].split(" "))+'_'+ki[1],ch)
    #break
print(count)

482


In [34]:
#Making sure to have even number of dialogues in each chat
for i in range(len(chats)):
    if(len(chats[i])%2!=0):
        chats[i]=chats[i][:-1]

In [52]:
#Without having context i.e not concatenating consecutive dialogue turns
inputs=[]
outputs=[]
for i in range(len(chats)):
    #for j in range(len(chats[i])):
        inputs.extend(chats[i][::2])
        outputs.extend(chats[i][1::2])

In [35]:
#FOR ENTIRE CONTEXT
inputs=[]
outputs=[]
for i in range(len(chats)):
    sent=''
    for j in range(0,len(chats[i]),2):
        #print(chats[i][j])
        sent+=chats[i][j]+" "
        inputs.append(sent.strip(" "))
        outputs.append(chats[i][j+1].strip(" "))
        sent+=chats[i][j+1]+" "
print(len(inputs),len(outputs))

809 809


In [26]:
#Trainset creation
ndf=pd.DataFrame()
ndf["inputs"]=inputs
ndf["outputs"]=outputs
df1=ndf[:-500]
df2=ndf[-500:]
df1=pd.concat([df1]*3, ignore_index=True)
df2=pd.concat([df2]*3, ignore_index=True)
df1=df1.sample(frac=1)
df2=df2.sample(frac=1)
df1.to_csv("train_data.csv",index=False)
df2.to_csv("val_data.csv",index=False)

In [36]:
#Trainset creation
ndf=pd.DataFrame()
ndf["inputs"]=inputs
ndf["outputs"]=outputs
ndf.to_csv("test_data.csv",index=False)

# Vocabulary Creation

In [42]:
from keras.preprocessing.text import Tokenizer
t=Tokenizer()
t.fit_on_texts(chats_complete)
#t.texts_to_sequences(inputs)
vocab=t.word_index
objects=list(set(objects))
#objects_norm=list(nkb['object'])
count=len(vocab)
for k,v in vocab.items():
    vocab[k]=v-1
vocab["<pad>"]=count
vocab["<unk>"]=count+1
vocab["<eos>"]=count+2
count=count+3
for obj in objects:
    vocab[obj]=count
    count=count+1
#Dict to json
import json
with open('vocabulary.json', 'w') as fp:
    json.dump(vocab, fp)